In [1]:
from carbon import CarbonSimulatorUI
from carbon.simulators.carbon_simulator import (
    __version__ as uiversion,
    __date__ as uidate,
)
from jupyformat import *

In [2]:
print("[carbon_simulator] version", uiversion, uidate)
jp()

[carbon_simulator] version 0.9.10.1 12/Nov/2022


# Carbon Simulation - Test 23

In [3]:
Sim = CarbonSimulatorUI(pair="ETHUSDC", raiseonerror=True, verbose=False)
Sim

CarbonSimulatorUI(<0 positions, 0 trades>, pair='ETHUSDC')

In [4]:
help(Sim.add_sgl_pos)

Help on method add_sgl_pos in module carbon.simulators.carbon_simulator:

add_sgl_pos(tkn: str, amt: Any, p_lo: Any, p_hi: Any, pair: str = None) -> Dict[str, Any] method of carbon.simulators.carbon_simulator.CarbonSimulatorUI instance
    adds a single position for sale of tkn
    
    :tkn:       the token that is being added to the position, eg "ETH"; it is the token being sold*
    :amt:       the amount of `tkn` that is added to the position
    :p_lo:      the lower end* of the range, quoted in the currency of the pair
    :p_hi:      ditto upper end*
    :pair:      the token pair to which the position corresponds, eg "ETHUSD"*
    
    *p_lo, p_hi are interchangeable, the code deals with sorting them to pmin, pmax



We assert that we can not add an unrelated position to this pair, either using an existing token like LINK, or a token like DNE (does not exist).

In [5]:
#Sim.add_sgl_pos("LINK", 100, 2000, 2000)

We set up a single curve where the AMM sells ETH against USDC at 2000

In [6]:
Sim.add_sgl_pos("ETH", 10, 2000, 3000)
Sim.add_sgl_pos("USDC", 10000, 1000, 500)
Sim

CarbonSimulatorUI(<2 positions, 0 trades>, pair='ETHUSDC')

In [7]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,10.000000,10.000000,ETH,2000.000000,3000.000000,USDC per ETH,0
1,1,ETHUSDC,USDC,10000.000000,10000.000000,USDC,1000.000000,500.000000,USDC per ETH,1


We are trying to make the AMM buy ETH. That fails because it does not have any USD.

In [8]:
Sim.amm_sells("ETH", 0.0001)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,0.0,0,0,route #0,False,0.0001,ETH,0.2,USDC,ETHUSDC,0,1,2000.00367,USDC per ETH
0,0,0,None,AMM sells 0ETH buys 0USDC,True,0.0001,ETH,0.2,USDC,ETHUSDC,[0],1,2000.00367,USDC per ETH


In [9]:
Sim.amm_sells("ETH", 1)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,1.0,1,0,route #0,False,1.0,ETH,2037.394291,USDC,ETHUSDC,0,1,2037.394291,USDC per ETH
0,1,1,None,AMM sells 1ETH buys 2037USDC,True,1.0,ETH,2037.394291,USDC,ETHUSDC,[0],1,2037.394291,USDC per ETH


In [10]:
Sim.amm_sells("USDC", 0.001)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,2.0,2,0,route #1,False,0.001,USDC,0.000001,ETH,ETHUSDC,1,1,1000.0,USDC per ETH
0,2,2,None,AMM sells 0USDC buys 0ETH,True,0.001,USDC,0.000001,ETH,ETHUSDC,[1],1,1000.0,USDC per ETH


In [11]:
Sim.amm_sells("USDC", 10)["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,3.0,3,0,route #1,False,10.0,USDC,0.010003,ETH,ETHUSDC,1,1,999.707048,USDC per ETH
0,3,3,None,AMM sells 10USDC buys 0ETH,True,10.0,USDC,0.010003,ETH,ETHUSDC,[1],1,999.707048,USDC per ETH


In [12]:
ji("We see below that the first (small) trade is essentially at the boundary of the range (smaller number for selling ETH on the ETH upside, bigger number for selling USD and therefore buying ETH on the downside)")

In [13]:
Sim.state()["trades"]

,uid,id,sub_id,note,aggr,amt1,tkn1,amt2,tkn2,pair,route_indexes,num_trades,price_avg,p_unit
0,0.0,0,0,route #0,False,0.0001,ETH,0.200000,USDC,ETHUSDC,0,1,2000.00367,USDC per ETH
0,0,0,None,AMM sells 0ETH buys 0USDC,True,0.0001,ETH,0.200000,USDC,ETHUSDC,[0],1,2000.00367,USDC per ETH
0,1.0,1,0,route #0,False,1.0000,ETH,2037.394291,USDC,ETHUSDC,0,1,2037.394291,USDC per ETH
0,1,1,None,AMM sells 1ETH buys 2037USDC,True,1.0000,ETH,2037.394291,USDC,ETHUSDC,[0],1,2037.394291,USDC per ETH
0,2.0,2,0,route #1,False,0.0010,USDC,0.000001,ETH,ETHUSDC,1,1,1000.0,USDC per ETH
0,2,2,None,AMM sells 0USDC buys 0ETH,True,0.0010,USDC,0.000001,ETH,ETHUSDC,[1],1,1000.0,USDC per ETH
0,3.0,3,0,route #1,False,10.0000,USDC,0.010003,ETH,ETHUSDC,1,1,999.707048,USDC per ETH
0,3,3,None,AMM sells 10USDC buys 0ETH,True,10.0000,USDC,0.010003,ETH,ETHUSDC,[1],1,999.707048,USDC per ETH


In [14]:
Sim.state()["orders"]

,id,pair,tkn_name,y_int,y,y_unit,p_start,p_end,p_unit,linked_to_id
0,0,ETHUSDC,ETH,10.000000,8.999900,ETH,2000.000000,3000.000000,USDC per ETH,0
1,1,ETHUSDC,USDC,10000.000000,9989.999000,USDC,1000.000000,500.000000,USDC per ETH,1
